In [1]:
%gui qt6
import pandas as pd
import numpy as np
import pyqtgraph as pg
import PyQt6 as p6
import os
pd.options.mode.chained_assignment = None

In [2]:
Si_nk = pd.read_csv('../resources/Si_nk.txt', sep='\t').drop('Unnamed: 3', axis=1)

In [3]:
Si_nk

,wvl,n,k
0,0.250,1.66361,3.662700
1,0.255,1.69017,3.848640
2,0.260,1.75387,4.087200
3,0.265,1.87822,4.372220
4,0.270,2.08515,4.695810
...,...,...,...
146,0.980,3.58348,0.000520
147,0.985,3.58162,0.000479
148,0.990,3.57980,0.000441
149,0.995,3.57801,0.000405


In [4]:
def read_completeEASE(file):
    df = pd.read_csv(file, sep='\t', header=None)[::-1].reset_index(drop=True)
    return df

def normalize_data(df):
    _df = df.copy(deep=True)
    _df.loc[:,1][_df.loc[:,1]<0] = _df.loc[:,1] + 360
    _df.loc[:,2][_df.loc[:,2]<0] = _df.loc[:,2] + 360
    return _df

In [ ]:
Si_psi_delta_60 = normalize_data(read_completeEASE(os.path.join('..', 'resources', 'Si_psi_delta_60ang.csv')))

In [13]:
def Snells_law(ni, nt, thetai):
    thetat = np.arcsin(ni/nt * np.sin(thetai))
    return thetat

def rp(ni, nt, thetai):
    thetat = Snells_law(ni, nt, thetai)
    return (nt*np.cos(thetai) - ni*np.cos(thetat))/(nt*np.cos(thetai) + ni*np.cos(thetat))

def rs(ni, nt, thetai):
    thetat = Snells_law(ni, nt, thetai)
    return (ni*np.cos(thetai) - nt*np.cos(thetat))/(ni*np.cos(thetai) + nt*np.cos(thetat))

def eV_to_um(eV):
    return 1239.8/eV

def um_to_eV(um):
    return 1239.8/um

In [15]:
def nk_plot(wvl, n, k):
    plt = pg.plot()
    plt.showGrid(x=True, y=True)
    plt.addLegend()
    plt.plot(wvl, n, pen=pg.mkPen('red', width=1.5), name='n')
    plt.plot(wvl, k, pen=pg.mkPen('blue', width=1.5), name='k')
    plt.setLabel('bottom', 'Wavelength', units='μm')
    plt.setWindowTitle('Reflectance plot')
    plt.setBackground('w')

def reflectance_plot(ni, nt, squared: bool = True, degrees: bool = True):
    plt = pg.plot()
    angle_space = np.radians(np.arange(0,90.1, 0.1))
    plt.showGrid(x=True, y=True)
    plt.addLegend()
    r_p = rp(ni, nt, angle_space)
    r_s = rs(ni, nt, angle_space)
    brewster_angle_x = angle_space[np.argmin(rp(ni, nt, angle_space)**2)]

    if degrees:
        angle_label = np.rad2deg(angle_space)
        brewster_angle_x = np.rad2deg(brewster_angle_x)
        plt.setLabel('bottom', 'Degrees', units='°')
    else:
        angle_label = angle_space
        plt.setLabel('bottom', 'Radians')

    if squared:
        plt.plot(angle_label, r_p**2, pen=pg.mkPen('red', width=1.5), name='R_p')
        plt.plot(angle_label, r_s**2, pen=pg.mkPen('blue', width=1.5), name='R_s')
    else:
        plt.plot(angle_label, r_p, pen=pg.mkPen('red', width=1.5), name='r_p')
        plt.plot(angle_label, r_s, pen=pg.mkPen('blue', width=1.5), name='r_s')

    annotation = pg.TextItem(f'{brewster_angle_x.round(2)}', color='red')
    annotation.setPos(brewster_angle_x,-0.01)
    annotation_dot = pg.ScatterPlotItem(size=5, brush='black')
    annotation_dot.addPoints([{'pos': [brewster_angle_x,0]}])
    plt.addItem(annotation_dot)
    plt.addItem(annotation)
    plt.setWindowTitle('Reflectance plot')
    plt.setBackground('w')


def psi_delta_plot(wvl, psi, delta):
    plt = pg.plot()
    plt.showGrid(x=True, y=True)
    plt.addLegend()
    plt.plot(wvl, psi, pen=pg.mkPen('red', width=1.5), name='<font>&Psi;</font>')
    plt.plot(wvl, delta, pen=pg.mkPen('blue', width=1.5), name='<font>&Delta;</font>')
    plt.setLabel('bottom', 'Wavelength', units='μm')
    plt.setWindowTitle('Psi Delta plot')
    plt.setBackground('w')



In [16]:
nk_plot(Si_nk['wvl'], Si_nk['n'], Si_nk['k'])

In [17]:
reflectance_plot(1, Si_nk['n'].iloc[51])

In [18]:
psi_delta_plot(Si_psi_delta_60.iloc[:,0], Si_psi_delta_60.iloc[:,1], Si_psi_delta_60.iloc[:,2])

In [ ]:
import pyqtgraph.examples 
  
# run this examples 
pyqtgraph.examples.run() 